<a href="https://colab.research.google.com/github/dariiimo/nlp2021/blob/main/lab5/Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np, pandas as pd, tensorflow as tf, requests as rqst, io
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers

rnd = np.random.randint

folder = '/content/drive/MyDrive/нлп/'

## На основі підготовлених у попередній лабораторній роботі речень кожного з учасників створити загальний набір даних розмірністю (100, n), де n -кількість учасників команди, та відповідні мітки класів до них


In [8]:
RL_file = open(folder+'sentences.txt')
RL_sentences = RL_file.read().split('. ')[:-1]

DARINA_file = open(folder+'100.txt', encoding='windows-1251')
DARINA_sentences = DARINA_file.read().split('. ')[:-1]

max_tokens = 10000

sentences = RL_sentences+DARINA_sentences
tokens_count = 0
for sent in sentences:
  tokens_count+=len(sent.split())
avg_tokens = round(tokens_count/len(sentences))
avg_tokens

RL_sentences

['Training a virtual agent to outperform human players can teach us how to optimize different processes in a variety of different and exciting subfields',
 'This is what Google DeepMind did with its popular AlphaGo, which beat the strongest Go player in history and scored a goal that was considered impossible at the time.In this article, we will develop an AI agent that is able to learn how to play the popular game Snake from scratch',
 'To do it, we implement a Deep Reinforcement Learning algorithm using both Keras on top of Tensorflow and PyTorch (both versions are available, you can choose the one you prefer)',
 'This approach consists in the interaction between two components: an environment (the game itself) and an agent (Snake)',
 'The agent collects information about its current state (we will see later what this means) and performs an action accordingly',
 'The environment rewards or punishes the agent based on the performed action',
 'Over time, the agent learns what actions m

In [9]:
sent_df = []

for sent in RL_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Reinforcement Learning'})

for sent in DARINA_sentences:
  sent_df.append({'Sentence':sent, 'Target':'Neural network module for recognizing brain diseases'})

sent_df = pd.DataFrame(data=sent_df, columns=['Sentence', 'Target'])

sent_df['Label'] = sent_df['Target']
sent_df.replace({'Label':{'Reinforcement Learning':0, 'Neural network module for recognizing brain diseases':1}}, inplace=True)
sent_df

,Sentence,Target,Label
0,Training a virtual agent to outperform human p...,Reinforcement Learning,0
1,This is what Google DeepMind did with its popu...,Reinforcement Learning,0
2,"To do it, we implement a Deep Reinforcement Le...",Reinforcement Learning,0
3,This approach consists in the interaction betw...,Reinforcement Learning,0
4,The agent collects information about its curre...,Reinforcement Learning,0
...,...,...,...
211,This includes balancing the prospective benefi...,Neural network module for recognizing brain di...,1
212,"Users report their symptoms into the app, whic...",Neural network module for recognizing brain di...,1
213,Most common users could be people with serious...,Neural network module for recognizing brain di...,1
214,Body scans can spot cancer and vascular diseas...,Neural network module for recognizing brain di...,1


#### Tokenization&Embedding

In [10]:
text_vectorizer = TextVectorization(max_tokens=max_tokens, 
                                    standardize="lower_and_strip_punctuation", 
                                    split="whitespace", 
                                    ngrams=None, 
                                    output_mode="int",
                                    output_sequence_length=avg_tokens, 
                                    pad_to_max_tokens=True)

text_vectorizer.adapt(sent_df['Sentence'])

text_vectorizer(sent_df['Sentence'])

<tf.Tensor: shape=(216, 23), dtype=int64, numpy=
array([[ 108,    6,  340, ...,  120,    4, 1214],
       [  27,    8,   47, ...,    6,   76,    9],
       [   3,  152,   26, ...,  618,   22,  216],
       ...,
       [  86,  314,  342, ...,    7,   64,  126],
       [1387,  794,   20, ..., 1159,    0,    0],
       [ 244,    2,  119, ...,  446,    8,    2]])>

In [11]:
embedding = layers.Embedding(input_dim=max_tokens, 
                             output_dim=128, 
                             embeddings_initializer="uniform", 
                             input_length=avg_tokens) 

embedding(text_vectorizer(sent_df['Sentence']))

<tf.Tensor: shape=(216, 23, 128), dtype=float32, numpy=
array([[[-0.03388479, -0.04768075,  0.03078211, ..., -0.03321211,
          0.04761798,  0.01684901],
        [ 0.04252862,  0.01630613, -0.03705528, ...,  0.02445502,
          0.03085256,  0.01245614],
        [-0.02924492, -0.03447485,  0.03112706, ..., -0.0067829 ,
         -0.02896518, -0.01596857],
        ...,
        [ 0.04709807, -0.02181363,  0.03530426, ...,  0.00954552,
         -0.00872532,  0.01410956],
        [-0.03746644,  0.01694191, -0.03738953, ...,  0.02119069,
          0.03472631,  0.0356879 ],
        [ 0.01162535,  0.04556067, -0.01055593, ...,  0.00147337,
          0.03196568, -0.01230179]],

       [[-0.0328695 , -0.04637541,  0.04104748, ...,  0.00663266,
         -0.03744509, -0.04612912],
        [-0.04165722,  0.03011524, -0.02073898, ...,  0.0050559 ,
          0.01002469,  0.00291776],
        [ 0.01242688,  0.00574857,  0.00096856, ..., -0.03897325,
         -0.04034108,  0.03804047],
        ...

#### Train/Test Split

In [12]:
from sklearn.model_selection import train_test_split
train_split, test_split = train_test_split(sent_df, train_size=0.8, test_size=0.2)
train_split

,Sentence,Target,Label
60,The computer employs trial and error to come u...,Reinforcement Learning,0
157,Support vector machine was also applied in can...,Neural network module for recognizing brain di...,1
147,"The lack of data, complexity and heterogeneity...",Neural network module for recognizing brain di...,1
135,"By sifting out healthy patients, the system al...",Neural network module for recognizing brain di...,1
23,"A state can be its position, its speed, or wha...",Reinforcement Learning,0
...,...,...,...
166,Although there is wisdom of excessive prospect...,Neural network module for recognizing brain di...,1
155,Some solutions to recognize potential biases a...,Neural network module for recognizing brain di...,1
109,And AI researchers still haven’t figured out h...,Reinforcement Learning,0
99,“We think RL can be leveraged to advance the s...,Reinforcement Learning,0


## Вирішити завдання мультикласової класифікації на створеному наборі даних із використанням методів: USE та USE-FT, здійснити порівняння моделей

In [13]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  -----
  y_true = true labels in the form of a 1D array
  y_pred = predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


### Naive Bayes Model

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_NB = Pipeline([
                    ("tfidf", TfidfVectorizer()), 
                    ("clf", MultinomialNB()) 
])

# Fit the pipeline to the training data
model_NB.fit(train_split['Sentence'], train_split['Label'])
baseline_score = model_NB.score(test_split['Sentence'], test_split['Label'])
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 77.27%


### Sentence Encoder

In [15]:
import tensorflow_hub as hub
url="https://tfhub.dev/google/universal-sentence-encoder/4"

sentence_encoder_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

model_use = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_use.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_use.summary()


Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [17]:
model_use_history=model_use.fit(train_split['Sentence'],train_split['Label'],epochs=5,
                        validation_data=(test_split['Sentence'],test_split['Label']))

Epoch 1/5
6/6 [==============================] - 5s 137ms/step - loss: 0.6822 - accuracy: 0.6337 - val_loss: 0.6649 - val_accuracy: 0.7955
Epoch 2/5
6/6 [==============================] - 0s 33ms/step - loss: 0.6359 - accuracy: 0.9186 - val_loss: 0.6294 - val_accuracy: 0.8864
Epoch 3/5
6/6 [==============================] - 0s 30ms/step - loss: 0.5900 - accuracy: 0.9535 - val_loss: 0.5906 - val_accuracy: 0.8864
Epoch 4/5
6/6 [==============================] - 0s 29ms/step - loss: 0.5399 - accuracy: 0.9593 - val_loss: 0.5454 - val_accuracy: 0.9091
Epoch 5/5
6/6 [==============================] - 0s 27ms/step - loss: 0.4860 - accuracy: 0.9593 - val_loss: 0.4999 - val_accuracy: 0.9318


### Sentence Encoder with fine tuning

In [18]:
tf_hub_embedding_layer = hub.KerasLayer(url,input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=True, 
                                        name="USE-FT")

model_use_ft = tf.keras.Sequential([
  tf_hub_embedding_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(1, activation="sigmoid")
], name="model_use_ft")

# Compile model
model_use_ft.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


model_use_ft.summary()


Model: "model_use_ft"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE-FT (KerasLayer)         (None, 512)               256797824 
                                                                 
 dense_2 (Dense)             (None, 64)                32832     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 256,830,721
Non-trainable params: 0
_________________________________________________________________


In [19]:
model_use_ft_history=model_use_ft.fit(train_split['Sentence'],train_split['Label'],epochs=10,
                        validation_data=(test_split['Sentence'],test_split['Label']))


Epoch 1/10
6/6 [==============================] - 9s 372ms/step - loss: 0.6511 - accuracy: 0.7035 - val_loss: 0.6372 - val_accuracy: 0.6591
Epoch 2/10
6/6 [==============================] - 2s 265ms/step - loss: 0.5280 - accuracy: 0.9244 - val_loss: 0.5326 - val_accuracy: 0.7955
Epoch 3/10
6/6 [==============================] - 2s 264ms/step - loss: 0.3978 - accuracy: 0.9535 - val_loss: 0.4351 - val_accuracy: 0.7955
Epoch 4/10
6/6 [==============================] - 2s 267ms/step - loss: 0.2584 - accuracy: 0.9884 - val_loss: 0.3324 - val_accuracy: 0.9091
Epoch 5/10
6/6 [==============================] - 2s 272ms/step - loss: 0.1566 - accuracy: 0.9884 - val_loss: 0.3168 - val_accuracy: 0.8182
Epoch 6/10
6/6 [==============================] - 2s 269ms/step - loss: 0.0878 - accuracy: 0.9884 - val_loss: 0.2030 - val_accuracy: 0.9318
Epoch 7/10
6/6 [==============================] - 2s 266ms/step - loss: 0.0483 - accuracy: 1.0000 - val_loss: 0.1799 - val_accuracy: 0.9318
Epoch 8/10
6/6 [====

## Здійснити порівняння ефективності моделей

### NB

In [20]:
calculate_results(test_split['Label'], model_NB.predict(test_split['Sentence']))

{'accuracy': 77.27272727272727,
 'f1': 0.7703745529832488,
 'precision': 0.8538961038961038,
 'recall': 0.7727272727272727}

### USE

In [21]:
model_use_preds=model_use.predict(test_split['Sentence'])
model_use_preds=tf.squeeze(tf.round(model_use_preds))
calculate_results(test_split['Label'],model_use_preds)

{'accuracy': 93.18181818181817,
 'f1': 0.9323533609247895,
 'precision': 0.9415584415584416,
 'recall': 0.9318181818181818}

### USE-FT

In [22]:
model_use_ft_preds=model_use_ft.predict(test_split['Sentence'])
model_use_ft_preds=tf.squeeze(tf.round(model_use_ft_preds))
calculate_results(test_split['Label'],model_use_ft_preds)

{'accuracy': 93.18181818181817,
 'f1': 0.9323533609247895,
 'precision': 0.9415584415584416,
 'recall': 0.9318181818181818}